# Tutorial: Real-Time Anomaly Detection with the Heston Model

This notebook demonstrates how to use the DRR framework for real-time anomaly detection on financial data simulated with the Heston model.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from drr_framework.benchmarks import BenchmarkSystems
from drr_framework.realtime import RealTimeDRR

plt.style.use('seaborn-v0_8-whitegrid')

## 1. Generate Heston Model Data

First, we generate a dataset using the Heston model, which simulates stock price and its stochastic volatility.

In [ ]:
duration = 252  # 1 year of trading days
dt = 1/252
t, heston_data = BenchmarkSystems.generate_heston_data(duration=duration, dt=dt)
stock_price = heston_data[:, 0]
volatility = heston_data[:, 1]

## 2. Visualize the Data

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

color = 'tab:blue'
ax1.set_xlabel('Time (days)')
ax1.set_ylabel('Stock Price', color=color)
ax1.plot(t, stock_price, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Volatility', color=color)
ax2.plot(t, volatility, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
plt.title('Heston Model: Stock Price and Volatility')
plt.show()

## 3. Real-Time Anomaly Detection

Now, we'll process the data in a simulated real-time stream and detect anomalies in the volatility.

In [ ]:
window_size = 30
n_variables = 2
threshold = 0.02 # Anomaly threshold for volatility

rt_drr = RealTimeDRR(window_size=window_size, n_variables=n_variables, threshold=threshold)

results = []
for i in range(len(heston_data)):
    result = rt_drr.process_data_point(heston_data[i])
    if result:
        results.append(result)

## 4. Visualize Anomaly Detection Results

In [ ]:
vol_depth = [r[1]['depth']['resonance_depth'] for r in results]
anomalies = [r[1]['anomaly'] for r in results]

plt.figure(figsize=(12, 6))
plt.plot(t[window_size-1:], vol_depth, label='Volatility Resonance Depth')
plt.axhline(y=threshold, color='r', linestyle='--', label='Anomaly Threshold')

anomaly_indices = np.where(anomalies)[0] + window_size - 1
plt.scatter(t[anomaly_indices], np.array(vol_depth)[np.where(anomalies)], color='red', zorder=5, label='Anomalies')

plt.title('Real-Time Anomaly Detection in Volatility')
plt.xlabel('Time (days)')
plt.ylabel('Resonance Depth (Rolling Std)')
plt.legend()
plt.show()